<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/CAFE_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:

from huggingface_hub import notebook_login

notebook_login()

In [3]:

from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:

import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
%%capture
!gdown --id 1EZTvcHo2x3cim4uSTkfMHoagipfQtLak
time.sleep(5)

!unzip /content/FESC_CaFE.zip

In [7]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/CaFE/CaFE_splits/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/CaFE/CaFE_splits/test.json",'r') as f:
    test_data = json.load(f)

In [8]:
train_data

{'09-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-4.wav',
  'length': 3.5731875,
  'emo': 'happiness'},
 '09-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-5.wav',
  'length': 3.8738125,
  'emo': 'happiness'},
 '04-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/04-J-1-6.wav',
  'length': 5.84675,
  'emo': 'happiness'},
 '02-J-1-1': {'wav': 'CaFE_48k/Happiness/Faible/02-J-1-1.wav',
  'length': 4.8348125,
  'emo': 'happiness'},
 '10-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/10-J-1-6.wav',
  'length': 4.38675,
  'emo': 'happiness'},
 '10-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/10-J-1-4.wav',
  'length': 4.6013125,
  'emo': 'happiness'},
 '02-J-1-3': {'wav': 'CaFE_48k/Happiness/Faible/02-J-1-3.wav',
  'length': 4.6151875,
  'emo': 'happiness'},
 '04-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/04-J-1-4.wav',
  'length': 4.626,
  'emo': 'happiness'},
 '09-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-6.wav',
  'length': 3.7488125,
  'emo': 'happiness'},
 '04-J-1-5': {'wav': 'CaFE_

In [9]:
test_data

{'11-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-6.wav',
  'length': 5.1439375,
  'emo': 'happiness'},
 '11-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-4.wav',
  'length': 4.9079375,
  'emo': 'happiness'},
 '11-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-5.wav',
  'length': 4.6163125,
  'emo': 'happiness'},
 '11-J-1-1': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-1.wav',
  'length': 4.2954375,
  'emo': 'happiness'},
 '11-J-1-2': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-2.wav',
  'length': 4.0303125,
  'emo': 'happiness'},
 '11-J-1-3': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-3.wav',
  'length': 4.2556875,
  'emo': 'happiness'},
 '12-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-6.wav',
  'length': 5.267,
  'emo': 'happiness'},
 '12-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-5.wav',
  'length': 5.34375,
  'emo': 'happiness'},
 '12-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-4.wav',
  'length': 5.4876875,
  'emo': 'happiness'},
 '12-J-1-1': {'wav': 'CaF

In [10]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)


In [11]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [12]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
})

In [13]:
def add_base_path(example):
    example['wav'] = '/content/CaFE/' + example['wav']  # Adjust column name if it's not 'wav'
    return example

dataset = dataset.map(add_base_path, batched=False)
dataset['train'][0]

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

{'id': '09-J-1-4',
 'wav': '/content/CaFE/CaFE_48k/Happiness/Faible/09-J-1-4.wav',
 'length': 3.5731875,
 'emo': 'happiness'}

In [14]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [15]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
})

In [16]:
dataset['train'][0].keys()

dict_keys(['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'])

In [17]:
import torch
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor, TrainingArguments, Trainer
!pip install datasets --upgrade


In [18]:
# Define labels and mappings for your dataset
labels = ['anger', 'happiness', 'neutral', 'sadness']
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

In [19]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# Filter out examples with labels that are not in label2id
dataset = dataset.filter(lambda x: x['emo'] in label2id.keys())
# Preprocess function to convert audio to model-compatible inputs
def preprocess_function(batch):
    # Process audio array into input values for the model
    batch["input_values"] = processor(batch["audio_array"], sampling_rate=16000).input_values[0]
    # Convert label to integer id
    batch["labels"] = label2id[batch["emo"]]
    return batch

# Map preprocess function to the dataset
dataset = dataset.map(preprocess_function, remove_columns=["audio_array", "sample_rate", "wav"])


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

Filter:   0%|          | 0/156 [00:00<?, ? examples/s]

Filter:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

In [20]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer, AutoFeatureExtractor

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)


In [22]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,  # Your dataset
    batch_size=32,   # Or any other suitable batch size
    collate_fn=collate_fn,  # Custom collate function
    shuffle=True
)


In [24]:
import torch
from sklearn.metrics import recall_score
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Sort batch by sequence length
    batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)

    # Convert input values to tensors and labels to integers
    inputs = [torch.tensor(example["input_values"]) for example in batch]
    labels = [int(example["label"]) for example in batch]

    # Pad the sequences to the same length
    inputs = pad_sequence(inputs, batch_first=True)

    # Return input sequences and labels as a dictionary
    return {"input_values": inputs, "label": torch.tensor(labels)}

def train(model, train_loader, optimizer, loss_fn, device):
    model.train()

    total_predictions = 0
    correct_predictions = 0
    total_train_loss = 0

    # These lists will hold actual labels, predictions, and confidence scores for evaluation
    all_actual_labels = []
    all_predictions = []
    train_confidence_scores = []

    # Loop over the DataLoader (batches)
    for batch in train_loader:
        inputs, labels = batch['input_values'].to(device), batch['label'].to(device)

        optimizer.zero_grad()  # Reset gradients

        # Forward pass through the model
        outputs = model(inputs)
        outputs = outputs.logits  # Assuming the model outputs logits

        # Compute the loss
        loss = loss_fn(outputs, labels)
        loss.backward()  # Backpropagation

        optimizer.step()  # Update the model parameters

        # Track batch-wise statistics
        _, predicted = torch.max(outputs, 1)  # Get predicted labels (class with max logit)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        total_train_loss += loss.item()

        # Store actual labels and predicted labels for evaluation
        all_actual_labels.extend(labels.tolist())
        all_predictions.extend(predicted.tolist())

        # Compute confidence scores (probabilities for each class)
        max_values, max_indices = torch.softmax(outputs, dim=1).max(dim=1)
        probability_label_tuples = [(max_value.item(), max_index.item()) for max_value, max_index in zip(max_values, max_indices)]
        train_confidence_scores.extend(probability_label_tuples)

    # Compute the average loss and accuracy over the entire dataset
    total_train_loss /= len(train_loader)
    train_accuracy = correct_predictions / total_predictions

    # Calculate recall scores (both weighted and unweighted)
    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_actual_labels, all_predictions, average='weighted')

    # Return performance metrics
    return {
        "unweighted_recall": unweighted_recall,
        "weighted_recall": weighted_recall,
        "train_accuracy": train_accuracy,
        "train_loss": total_train_loss,
        "train_confidence_scores": train_confidence_scores
    }
